In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [2]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment

In [3]:
manual_path = "webpages/manually_selected"

folder_path = {
    "coursera": manual_path + "/coursera",
    "edX": manual_path + "/edX",
    "general": manual_path + "/general",
}

In [4]:
def make_path(folder, positive=True):
    middle = "positive" if positive else "negative"
    path = "{}/{}/urls.txt".format(folder_path[folder], middle)
    return path

In [5]:
def urls_list(folder, positive=True):
    path = make_path(folder, positive)
    
    
    with open(path, 'r') as f:
        urls = list(l.strip() for l in f if l[0] != "#")
    f.close()
    
    return urls



In [6]:
positives = urls_list("coursera")[:5]
positives

FileNotFoundError: [Errno 2] No such file or directory: 'webpages/manually_selected/coursera/positive/urls.txt'

In [ ]:
negatives = urls_list("coursera", positive=False)[:5]
negatives

In [ ]:
r = requests.get(positives[0])

In [ ]:
r.status_code

In [ ]:
soup = BeautifulSoup(r.text, "lxml")

In [ ]:
print(soup.html.body.prettify()[:1000])

In [ ]:
all_text = soup.html.body.findAll(text=True)
all_text[:10]

In [ ]:
def tag_visible(element):
    '''Keep only visible elements'''
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [ ]:
list(filter(tag_visible, all_text))[:15]

In [ ]:
import re

def extract_visible(soup):
    text = soup.html.body.findAll(text=True)
    s = ' '.join(filter(tag_visible, text))
    return re.sub("\s\s+" , " ", s) # remove all double spaces and tabs/newlines/etc.

In [ ]:
extract_visible(soup)[:1000]

In [ ]:
for positive in (True, False):
    for folder in ("coursera", "edX", "general"):
        urls = urls_list(folder, positive=positive)
        r = requests.get(urls[0])
        soup = BeautifulSoup(r.text, "lxml")
        
        print("={}, {}=".format(folder, "positive" if positive else "negative"))
        print()
        print(extract_visible(soup)[:1000])
        print()
            

In [ ]:
for positive in (True, False):
    urls = urls_list("edX", positive=positive)

    for url in urls[:5]:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "lxml")

        print("={}, {}=".format(folder, "positive" if positive else "negative"))
        print()
        print(extract_visible(soup)[:1000])